**IOC Case Study: ARP Spoofing (Host-Based – Layer 2 Attack)**

1. Precondition:
The attacker already has access to the local subnet—either by compromising an internal host, connecting physically to the network, or joining a poorly secured Wi-Fi.


Attacker Analogy: The Poisoned Directory
Imagine you're working in an office and need to send a document to the company director. You ask the front desk, “Where’s the director’s office?” and someone answers, “Room 204”—but they’re lying. That room is actually occupied by a malicious actor who intercepts your document, opens it, and possibly changes it before forwarding it on. This is ARP spoofing: poisoning the “directory” your device uses to find other machines on the network, so your traffic goes to the wrong destination first.

Address Resolution Protocol (ARP) is used to resolve an IP address (Layer 3) to its corresponding MAC address (Layer 2) within a local area network (LAN).

**2. Source of IOC (Telemetry Origin)**

Signature Insight: Weak or Absent Initial IOC Visibility
ARP spoofing is one of the few attacks that often lacks a distinct, standalone indicator of compromise at the outset. Because Address Resolution Protocol (ARP) operates at Layer 2 and is unauthenticated by design, most default system configurations do not generate reliable alerts when ARP behavior is malicious. Detection typically relies on secondary behavioral indicators, such as unusual traffic patterns, failed routing behavior, or forensic packet analysis—not a single definitive telemetry artifact. This makes ARP spoofing a high-risk, low-visibility threat, especially in flat networks without endpoint protection tuned for Layer 2 monitoring.

Windows Event Logs (Security + System): Unusual ARP activity is not logged by default in standard Windows Event Logs. Detecting ARP spoofing typically requires enabling special packet-level auditing, using third-party EDR tools, or capturing network traffic directly. Therefore, unless custom auditing or network monitoring is in place, the Windows event logs alone may not reveal ARP-related anomalies.

Packet Capture (PCAP): duplicated or suspicious ARP reply frames; multiple IPs mapping to same MAC

NetFlow Data: Sudden traffic redirection or volume increases between non-gateway internal endpoints may indicate unauthorized routing behavior, prompting further inspection for ARP spoofing.
If you know endpoint A usually talks to gateway R, and endpoint B talks to gateway R, but now A and B are suddenly talking to each other over unusual volumes, or A is suddenly sending all of its traffic to B, and B isn't a gateway, DNS server, or proxy, then…

“Why is all of A’s traffic suddenly routed through B?”

That’s when you go looking for Layer 2 manipulation, such as ARP spoofing, because B may have told A, "I’m your default gateway."

Volatile Memory (via memory dump): attacker tools in use such as arpspoof, ettercap, or raw socket usage

**3. Triage Framework Declaration: Host System Activity**
This IOC is primarily addressed using the Host System Triage Protocol, with pivots to Network Triage for validation.

Standard host triage toolkit includes:

Windows Event Logs (System, Security)

EDR telemetry (if present, check for local ARP table manipulation or packet crafting tools)

File system and registry inspection (e.g., presence of attacker tools, registry keys enabling promiscuous mode) Promiscuous Mode :
Promiscuous mode is a special setting on a network interface card (NIC)—the physical or virtual hardware component inside a device that connects it to a network. Normally, a NIC only processes frames addressed to its own MAC address, but in promiscuous mode, it accepts all network traffic it sees on the local segment, regardless of destination.

Tools like Wireshark rely on this capability to capture complete traffic, but they don’t have their own MAC address—rather, they instruct the NIC to enter promiscuous mode.

Attackers may enable promiscuous mode on compromised hosts to monitor nearby traffic silently. In Windows, doing this might require modifying registry keys or loading custom drivers—low-level software that allows the operating system to communicate with hardware like NICs.

Driver (Definition):
A driver is a small, specialized program that acts as a translator between the operating system and a hardware device. In this context, a network driver tells the OS how to interact with the network interface card (NIC), and a custom driver may extend or override this behavior (e.g., to enable promiscuous mode or raw packet capture).


Volatile memory capture (e.g., identify active network processes or raw socket usage). Raw Socket Usage:
Raw sockets allow software to bypass the standard TCP/IP stack and directly craft or receive packets at the IP or Ethernet level. Attackers use raw sockets to send forged ARP replies or manipulate packet headers manually. Their presence in memory is a red flag indicating low-level network manipulation outside of normal application behavior.


**4. OS Layer with Key Behaviors**
OS Layer	Activity
Layer 1: Process Execution	Attacker launches tool (e.g., arpspoof, responder, or Python script)
Layer 2: Startup & Persistence	May configure persistence via scheduled tasks or login script
Layer 3: Background Execution	Long-running spoofing tool silently listens and injects ARP replies
Layer 5: Monitoring/Detection Evasion	May disable detection logs or mask tool behavior
Layer 6: Network Communication	Injects forged ARP replies to manipulate internal traffic routing

**5. Cross-Layer Interaction Pivots**
From Layer	To Layer	Rationale
Layer 4 (Credentials) → Layer 1	Attacker authenticates or exploits to gain execution ability	
Layer 1 → Layer 6	Execution of spoofing utility that interacts with network interface	
Layer 6 → Layer 3	Attacker monitors background process behavior for sustained spoofing	
Layer 6 → Layer 2	Injects crafted ARP replies, emulating legitimate network comms	

**6. OSI Layer Relevance**
OSI Layer	Role in ARP Spoofing
Layer 2 (Data Link)	Core of the attack: manipulates IP to MAC address mappings using ARP
Layer 3 (Network)	Layer 2 (Data Link): Core of the attack—manipulates IP-to-MAC address mappings using the Address Resolution Protocol (ARP).

This corrected form reflects how ARP works:

A host says: “Who has IP address 192.168.1.1?”

The device with that IP replies: “I do—my MAC is aa:bb:cc:dd:ee:ff.”

An attacker fakes this reply, causing other devices to associate the attacker's MAC address with a legitimate IP (such as the gateway), which enables interception or redirection of traffic.
Layer 1 (Physical)	Must have access to LAN segment—ARP is not routed across subnets
Layer 7 (Application)	May harvest credentials from apps if MITM is successful (e.g., SMB, HTTP)

**7. Attacker Behavior Interpretation**
ARP spoofing is rarely the beginning of an attack. It represents a mid-stage tactical maneuver following successful initial access and lateral positioning. The attacker likely:

Already has host-level control (through credential theft or code execution)

Has network adapter access (Layer 6 in your OS model)

Seeks to reroute traffic for:

Credential harvesting

Session hijacking

Network reconnaissance

Pivoting into segmented subnets

The attacker abuses the trust and statelessness of ARP, which accepts replies without validation.

**8. Defender Action Summary**
Validate local ARP tables using arp -a for conflicting or duplicated entries

Use Wireshark to identify unsolicited ARP replies or frequent ARP broadcasts

Employ Dynamic ARP Inspection (DAI) at the switch level

Isolate compromised hosts and inspect memory or EDR logs for active spoofing tools

Verify default gateway MACs are not duplicated or altered

Cross-check routing behavior for mismatched destination–MAC associations

**9. Attacker Strategy Notes**
The attacker in this scenario is acting like a LAN-level imposter. Their strategy:

Infiltrate quietly: using phishing or exploits to compromise an endpoint

Blend in: use known tools (e.g., responder, mitm6) without tripping alerts

Position mid-path: silently reroute traffic to harvest or redirect without detection

Trigger lateral escalation: steal NTLM hashes, credentials, or relay sessions

They exploit weaknesses in flat or poorly segmented networks, relying on trust in local protocols (like ARP) and absence of Layer 2 logging.

**10. Summary of the Case (Key Takeaway)**
ARP Spoofing is not an isolated event—it is the tactical execution of a mid-stage attacker who has already compromised a host and now seeks to manipulate local traffic routing for stealthy gain. Detection requires both host-based awareness (tools, memory, adapter behavior) and network-based validation (packet anomalies, switch monitoring). It represents the attacker exploiting trust within the local broadcast domain.
